In [ ]:
import pandas as pd
from collections import defaultdict

tasks = ['nsp_choice', 'ordering', 'rst', 'segment', 'dissent', 'cloze']
models = ['bert', 'roberta', 'albert', 'gpt2', 'electra', 'bart', 't5', \
         'bert-avg', 'albert-avg', 'bert-large', 'bert-zh', 'bert-es', 'bert-de']

files = '.'

def read_csv(fname):
    layers = []; dev = []; test = []
    for line in open(fname).readlines()[1:]:
        data = line.split(', ')
        layers.append(int(data[0]))
        dev.append(float(data[1]))
        test.append(float(data[2]))
    df = pd.DataFrame()
    df['layers']=layers
    df['dev']=dev
    df['test']=test
    return df

In [ ]:
outputs = defaultdict(list)

for output in ['output1', 'output2', 'output3']:
    for task in tasks:
        if task == 'rst':
            for model in models:
                fname1 = f'{files}/{task}/{output}/{model}_nuclearity.csv'
                data1 = read_csv(fname1)
                for idx, row in data1.iterrows():
                    key = (task+'_nuclearity',model,row['layers'])
                    outputs[key].append(row['test'])
                
                fname2 = f'{files}/{task}/{output}/{model}_relation.csv'
                data2 = read_csv(fname2)
                for idx, row in data2.iterrows():
                    key = (task+'_relation',model,row['layers'])
                    outputs[key].append(row['test'])
                
        else:
            for model in models:
                fname = f'{files}/{task}/{output}/{model}.csv'
                try:
                    data = read_csv(fname)
                except:
                    print(fname)
                    continue
                for idx, row in data.iterrows():
                    key = (task,model,row['layers'])
                    outputs[key].append(row['test'])

In [ ]:
mean_std = {}
import numpy as np
for key in outputs.keys():
    mean = np.mean(outputs[key])
    std = np.std(outputs[key])
    mean_std[key] = [mean, std]

In [ ]:
unique = []
for key in mean_std.keys():
    unique.append(key[0])
updated_tasks = list(set(unique))

In [ ]:
key2col = {
    'nsp_choice': 'NSP with 4 MC',
    'ordering': 'Sent. Ordering',
    'dissent': 'Discourse Connective',
    'rst_nuclearity': 'Nuclearity',
    'rst_relation': 'Relation',
    'segment': 'EDU segmentation',
    'cloze': 'Cloze'
}

In [ ]:
!mkdir 'RESULT/MEAN'
!mkdir 'RESULT/STD'

In [ ]:
def to_file(fname, data):
    df = pd.DataFrame()
    df['Layer'] = data['Layer']
    #print(data)
    for key in key2col.values():
        if data[key]!= []:
            df[key] = data[key]
        else:
            df[key] = [None] * len(data['Layer'])
    df.to_csv(fname, index=False)

for model in models:
    data_mean = defaultdict(list)
    data_std =  defaultdict(list)
    for task in updated_tasks:
        if 'large' in model:
            data_mean['Layer'] = list(range(1, 25))
            data_std['Layer'] = list(range(1, 25))
            for i in range(1,25):
                key = (task, model, i)
                data_mean[key2col[task]].append(mean_std[key][0]) 
                data_std[key2col[task]].append(mean_std[key][1]) 
        else:
            data_mean['Layer'] = list(range(1, 13))
            data_std['Layer'] = list(range(1, 13))
            for i in range(1,13):
                try:
                    key = (task, model, i)
                    data_mean[key2col[task]].append(mean_std[key][0]) 
                    data_std[key2col[task]].append(mean_std[key][1]) 
                except:
                    continue
                    #print(key)
    to_file(f'RESULT/MEAN/{model}.csv', data_mean)
    to_file(f'RESULT/STD/{model}.csv', data_std)